<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Lab: Custom Training Loops in Keras**


Estimated time needed: **30** minutes


In this lab, you will learn to implement a basic custom training loop in Keras. 


## Objectives

By the end of this lab, you will: 

- Set up the environment 

- Define the neural network model 

- Define the Loss Function and Optimizer 

- Implement the custom training loop 

- Enhance the custom training loop by adding an accuracy metric to monitor model performance 

- Implement a custom callback to log additional metrics and information during training .


----


## Step-by-Step Instructions:


### Exercise 1: Basic custom training loop: 

#### 1. Set Up the Environment:

- Import necessary libraries. 

- Load and preprocess the MNIST dataset. 


In [1]:
!pip install tensorflow numpy

In [2]:
import os
import warnings
import tensorflow as tf 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


2024-10-04 10:46:37.282056: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 10:46:37.284485: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-04 10:46:37.288851: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-04 10:46:37.300691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 10:46:37.323904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

#### 2. Define the model: 

Create a simple neural network model with a Flatten layer followed by two Dense layers. 


In [3]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])


#### 3. Define Loss Function and Optimizer: 

- Use Sparse Categorical Crossentropy for the loss function. 
- Use the Adam optimizer. 


In [4]:
# Step 3: Define Loss Function and Optimizer

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam()


#### 4. Implement the Custom Training Loop: 

- Iterate over the dataset for a specified number of epochs. 
- Compute the loss and apply gradients to update the model's weights. 


In [5]:
# Step 4: Implement the Custom Training Loop

epochs = 1

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.496466636657715
Epoch 1 Step 200: Loss = 0.38989347219467163
Epoch 1 Step 400: Loss = 0.19525526463985443
Epoch 1 Step 600: Loss = 0.21734723448753357
Epoch 1 Step 800: Loss = 0.17296235263347626
Epoch 1 Step 1000: Loss = 0.4402461647987366
Epoch 1 Step 1200: Loss = 0.1704329252243042
Epoch 1 Step 1400: Loss = 0.26026657223701477
Epoch 1 Step 1600: Loss = 0.18984942138195038
Epoch 1 Step 1800: Loss = 0.21052001416683197


2024-10-04 10:47:31.296388: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Exercise 2: Adding Accuracy Metric:

Enhance the custom training loop by adding an accuracy metric to monitor model performance. 

#### 1. Set Up the Environment: 

Follow the setup from Exercise 1. 


In [6]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model: 
Use the same model as in Exercise 1. 


In [7]:
# Step 2: Define the Model

model = Sequential([ 
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define the loss function, optimizer, and metric: 

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer. 

- Add Sparse Categorical Accuracy as a metric. 


In [8]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with accuracy: 

Track the accuracy during training and print it at regular intervals. 


In [9]:
# Step 4: Implement the Custom Training Loop with Accuracy

epochs = 1  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.484708786010742 Accuracy = 0.125
Epoch 1 Step 200: Loss = 0.4006815254688263 Accuracy = 0.8383084535598755
Epoch 1 Step 400: Loss = 0.15291203558444977 Accuracy = 0.8697007298469543
Epoch 1 Step 600: Loss = 0.16443288326263428 Accuracy = 0.8839954137802124
Epoch 1 Step 800: Loss = 0.16334673762321472 Accuracy = 0.8956382870674133
Epoch 1 Step 1000: Loss = 0.4252116084098816 Accuracy = 0.9029095768928528
Epoch 1 Step 1200: Loss = 0.2274370789527893 Accuracy = 0.9089040160179138
Epoch 1 Step 1400: Loss = 0.27036580443382263 Accuracy = 0.9138785004615784
Epoch 1 Step 1600: Loss = 0.2485123872756958 Accuracy = 0.9169464111328125
Epoch 1 Step 1800: Loss = 0.20584796369075775 Accuracy = 0.9208946228027344


2024-10-04 10:48:27.796634: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Exercise 3: Custom Callback for Advanced Logging: 

Implement a custom callback to log additional metrics and information during training. 

#### 1. Set Up the Environment: 

Follow the setup from Exercise 1.


In [10]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0 

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model: 

Use the same model as in Exercise 1. 


In [11]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define Loss Function, Optimizer, and Metric: 

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer. 

- Add Sparse Categorical Accuracy as a metric. 


In [12]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with custom callback: 

Create a custom callback to log additional metrics at the end of each epoch.


In [13]:
from tensorflow.keras.callbacks import Callback

# Step 4: Implement the Custom Callback 
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')


In [14]:
# Step 5: Implement the Custom Training Loop with Custom Callback

epochs = 1
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')
    
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)
        
        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    
    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    
    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.437922954559326 Accuracy = 0.0625
Epoch 1 Step 200: Loss = 0.3804294168949127 Accuracy = 0.8367537260055542
Epoch 1 Step 400: Loss = 0.16401994228363037 Accuracy = 0.8675966262817383
Epoch 1 Step 600: Loss = 0.1914302408695221 Accuracy = 0.8825395107269287
Epoch 1 Step 800: Loss = 0.17660532891750336 Accuracy = 0.8949750065803528
Epoch 1 Step 1000: Loss = 0.45021748542785645 Accuracy = 0.902410089969635
Epoch 1 Step 1200: Loss = 0.1697186380624771 Accuracy = 0.9090081453323364
Epoch 1 Step 1400: Loss = 0.19080711901187897 Accuracy = 0.9143245816230774
Epoch 1 Step 1600: Loss = 0.2451283037662506 Accuracy = 0.9175515174865723
Epoch 1 Step 1800: Loss = 0.24690380692481995 Accuracy = 0.9215713739395142
End of epoch 1, loss: 0.03513997793197632, accuracy: 0.9234166741371155


2024-10-04 10:49:23.779734: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Exercise 4: Add Hidden Layers 

Next, you will add a couple of hidden layers to your model. Hidden layers help the model learn complex patterns in the data. 


In [15]:
from tensorflow.keras.layers import Input, Dense

# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation


In the above code: 

`Dense(64, activation='relu')` creates a dense (fully connected) layer with 64 units and ReLU activation function. 

Each hidden layer takes the output of the previous layer as its input.


### Exercise 5: Define the output layer 

Finally, you will define the output layer. Suppose you are working on a binary classification problem, so the output layer will have one unit with a sigmoid activation function. 


In [16]:
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

In the above code: 

`Dense(1, activation='sigmoid')` creates a dense layer with 1 unit and a sigmoid activation function, suitable for binary classification. 


### Exercise 6: Create the Model 

Now, you will create the model by specifying the input and output layers. 


In [17]:
model = Model(inputs=input_layer, outputs=output_layer)

In the above code: 

`Model(inputs=input_layer, outputs=output_layer)` creates a Keras model that connects the input layer to the output layer through the hidden layers. 


### Exercise 7: Compile the Model 

Before training the model, you need to compile it. You will specify the loss function, optimizer, and evaluation metrics. 


In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In the above code: 

`optimizer='adam'` specifies the Adam optimizer, a popular choice for training neural networks. 

`loss='binary_crossentropy'` specifies the loss function for binary classification problems. 

`metrics=['accuracy']` tells Keras to evaluate the model using accuracy during training. 


### Exercise 8: Train the Model 

You can now train the model on some training data. For this example, let's assume `X_train` is our training input data and `y_train` is the corresponding labels. 


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

# Step 1: Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 2: Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4974 - loss: 0.6984
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5090 - loss: 0.6929
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5764 - loss: 0.6831
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5358 - loss: 0.6848
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5881 - loss: 0.6815
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5675 - loss: 0.6838
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5978 - loss: 0.6801
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5649 - loss: 0.6771
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5871 - loss: 0.6738
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5639 - loss: 0.6799


In the above code: 

`X_train` and `y_train` are placeholders for your actual training data. 

`model.fit` trains the model for a specified number of epochs and batch size. 


### Exercise 9: Evaluate the Model 

After training, you can evaluate the model on test data to see how well it performs. 


In [20]:
# Example test data (in practice, use real dataset)
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4598 - loss: 0.7055
Test loss: 0.7127352952957153
Test accuracy: 0.4449999928474426


In the above code: 

`model.evaluate` computes the loss and accuracy of the model on test data. 

`X_test` and `y_test` are placeholders for your actual test data. 


## Practice Exercises 

### Exercise 1: Basic Custom Training Loop 

#### Objective: Implement a basic custom training loop to train a simple neural network on the MNIST dataset. 

#### Instructions: 

- Set up the environment and load the dataset. 

- Define the model with a Flatten layer and two Dense layers. 

- Define the loss function and optimizer. 

- Implement a custom training loop to iterate over the dataset, compute the loss, and update the model's weights. 


In [21]:
## Write your code here


<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function and Optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 

# Step 4: Implement the Custom Training Loop
for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()}')


### Exercise 2: Adding Accuracy Metric 

#### Objective: Enhance the custom training loop by adding an accuracy metric to monitor model performance. 

#### Instructions: 

1. Set up the environment and define the model, loss function, and optimizer. 

2. Add Sparse Categorical Accuracy as a metric. 

3. Implement the custom training loop with accuracy tracking.


In [22]:
## Write your code here


<details>
<summary>Click here for solution</summary><br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 

# Step 1: Set Up the Environment
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Training Loop with Accuracy Tracking
epochs = 5 
for epoch in range(epochs): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()} Accuracy = {accuracy_metric.result().numpy()}') 
    accuracy_metric.reset_states() 


### Exercise 3: Custom Callback for Advanced Logging 

#### Objective: Implement a custom callback to log additional metrics and information during training. 

#### Instructions: 

1. Set up the environment and define the model, loss function, optimizer, and metric. 

2. Create a custom callback to log additional metrics at the end of each epoch. 

3. Implement the custom training loop with the custom callback. 


<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Flatten 
from tensorflow.keras.callbacks import Callback 

# Step 1: Set Up the Environment
(x_train, y_train) = tf.keras.datasets.mnist.load_data() 
x_train = x_train / 255.0 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32) 

# Step 2: Define the Model
model = Sequential([ 
    Flatten(input_shape=(28, 28)), 
    Dense(128, activation='relu'), 
    Dense(10) 
]) 

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
optimizer = tf.keras.optimizers.Adam() 
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy() 

# Step 4: Implement the Custom Callback
class CustomCallback(Callback): 
    def on_epoch_end(self, epoch, logs=None): 
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}') 

# Step 5: Implement the Custom Training Loop with Custom Callback
custom_callback = CustomCallback() 

for epoch in range(5): 
    for x_batch, y_batch in train_dataset: 
        with tf.GradientTape() as tape: 
            logits = model(x_batch, training=True) 
            loss = loss_fn(y_batch, logits) 
        grads = tape.gradient(loss, model.trainable_weights) 
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) 
        accuracy_metric.update_state(y_batch, logits) 
    custom_callback.on_epoch_end(epoch, logs={'loss': loss.numpy(), 'accuracy': accuracy_metric.result().numpy()}) 
    accuracy_metric.reset_states() 


### Conclusion: 

Congratulations on completing this lab! You have now successfully created, trained, and evaluated a simple neural network model using the Keras Functional API. This foundational knowledge will allow you to build more complex models and explore advanced functionalities in Keras. 


Copyright © IBM Corporation. All rights reserved.
